In [ ]:
import heapq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import animation
from IPython.display import HTML
import time
import random

In [ ]:
def generate_maze(rows, cols):

    if rows % 2 == 0: rows += 1
    if cols % 2 == 0: cols += 1


    maze = np.ones((rows, cols), dtype=np.int8)


    start_r = random.randrange(1, rows, 2)
    start_c = random.randrange(1, cols, 2)
    maze[start_r, start_c] = 0  # Passage

    wall_list = []


    def add_frontiers(r, c):
        for dr, dc in [(-2,0), (2,0), (0,-2), (0,2)]:
            nr, nc = r + dr, c + dc
            if 0 < nr < rows and 0 < nc < cols and maze[nr, nc] == 1:
                wall_list.append((r, c, nr, nc))

    add_frontiers(start_r, start_c)

    while wall_list:
        idx = random.randint(0, len(wall_list)-1)
        r1, c1, r2, c2 = wall_list.pop(idx)

        if maze[r2, c2] == 1:

            adjacent_passages = 0
            for dr, dc in [(-2,0), (2,0), (0,-2), (0,2)]:
                ar, ac = r2 + dr, c2 + dc
                if 0 <= ar < rows and 0 <= ac < cols and maze[ar, ac] == 0:
                    adjacent_passages += 1
            if adjacent_passages == 1:

                mid_r, mid_c = (r1 + r2) // 2, (c1 + c2) // 2
                maze[mid_r, mid_c] = 0
                maze[r2, c2] = 0
                add_frontiers(r2, c2)

    return maze


maze = generate_maze(21, 21)

import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
plt.imshow(maze, cmap='binary')
plt.axis('off')
plt.show()


In [ ]:
def place_start_goal(maze):
    maze = maze.copy()
    rows, cols = maze.shape

    # Place Start
    maze[1, 1] = 2
    # Place Goal
    maze[rows-2, cols-2] = 3
    return maze


In [ ]:
maze = generate_maze(21, 21)
maze_with_sg = place_start_goal(maze)

# Custom colormap: 1=wall(black), 0=passage(white), 2=start(green), 3=goal(red)
cmap = mcolors.ListedColormap(['white', 'black', 'green', 'red'])
bounds = [0, 1, 2, 3, 4]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

plt.figure(figsize=(8,8))
plt.imshow(maze_with_sg, cmap=cmap, norm=norm)
plt.title("Maze with Start (green) and Goal (red)")
plt.axis('off')
plt.show()


In [ ]:
def get_neighbors(maze, r, c):

    rows, cols = maze.shape
    neighbors = []
    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:  # Up, Down, Left, Right
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols:
            if maze[nr, nc] in [0, 2, 3]:  # Passage, Start, or Goal
                neighbors.append((nr, nc))
    return neighbors


In [ ]:
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [ ]:
def a_star(maze, start, goal, visualize=True):
    rows, cols = maze.shape
    new_maze = maze.copy()
    visited = set()
    parent = {} 

    heap = []
    heapq.heappush(heap, (0 + manhattan_distance(start, goal), 0, start))

    while heap:
        f, g, (r, c) = heapq.heappop(heap)
        if (r, c) == goal:
            cur = goal
            while cur != start:
                if new_maze[cur] not in [2, 3]:
                    new_maze[cur] = 5 #blue
                cur = parent[cur]
            return new_maze, True

        if (r, c) in visited:
            continue
        visited.add((r, c))

        if new_maze[r, c] not in [2, 3]:  
            new_maze[r, c] = 4  #green

        for nr, nc in get_neighbors(maze, r, c):
            if (nr, nc) not in visited:
                parent[(nr, nc)] = (r, c)
                g_new = g + 1
                f_new = g_new + manhattan_distance((nr, nc), goal)
                heapq.heappush(heap, (f_new, g_new, (nr, nc)))

    return new_maze, False

astar_cmap = mcolors.ListedColormap(['white', 'black', 'orange', 'red', 'green', 'blue'])
astar_bounds = [0,1,2,3,4,5,6]
astar_norm = mcolors.BoundaryNorm(astar_bounds, astar_cmap.N)

maze = generate_maze(21, 21)
maze_with_sg = place_start_goal(maze)
start = (1, 1)
goal = (maze.shape[0]-2, maze.shape[1]-2)

solved_maze, found = a_star(maze_with_sg, start, goal)

plt.figure(figsize=(8,8))
plt.imshow(solved_maze, cmap=astar_cmap, norm=astar_norm)
plt.title(f"A* Path: {'Found' if found else 'Not Found'}")
plt.axis('off')
plt.show()


In [ ]:
def a_star_animated(maze, start, goal):
    rows, cols = maze.shape
    new_maze = maze.copy()
    visited = set()
    parent = {}

    heap = []
    heapq.heappush(heap, (0 + manhattan_distance(start, goal), 0, start))

    frames = [new_maze.copy()]

    while heap:
        f, g, (r, c) = heapq.heappop(heap)
        if (r, c) == goal:
            cur = goal
            while cur != start:
                if new_maze[cur] not in [2, 3]:
                    new_maze[cur] = 5  # Path=Blue
                    frames.append(new_maze.copy())
                cur = parent[cur]
            return frames, True

        if (r, c) in visited:
            continue
        visited.add((r, c))

        if new_maze[r, c] not in [2, 3]:
            new_maze[r, c] = 4  # Visited=Green
            frames.append(new_maze.copy())

        for nr, nc in get_neighbors(maze, r, c):
            if (nr, nc) not in visited:
                parent[(nr, nc)] = (r, c)
                g_new = g + 1
                f_new = g_new + manhattan_distance((nr, nc), goal)
                heapq.heappush(heap, (f_new, g_new, (nr, nc)))

    return frames, False


In [ ]:
maze = generate_maze(21, 21)
maze_with_sg = place_start_goal(maze)
start = (1, 1)
goal = (maze.shape[0]-2, maze.shape[1]-2)

frames, found = a_star_animated(maze_with_sg, start, goal)

# Color map: white, black, orange, red, green, blue
astar_cmap = mcolors.ListedColormap(['white', 'black', 'orange', 'red', 'green', 'blue'])
astar_bounds = [0,1,2,3,4,5,6]
astar_norm = mcolors.BoundaryNorm(astar_bounds, astar_cmap.N)

fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(frames[0], cmap=astar_cmap, norm=astar_norm)
plt.axis('off')
plt.title("A* Search Animation")

def animate(i):
    im.set_data(frames[i])
    return [im]

ani = animation.FuncAnimation(fig, animate, frames=len(frames), interval=30, blit=True)
plt.close(fig)
HTML(ani.to_jshtml())


In [ ]:
def a_star_animated_logging(maze, start, goal):
    rows, cols = maze.shape
    new_maze = maze.copy()
    visited = set()
    parent = {}

    heap = []
    heapq.heappush(heap, (0 + manhattan_distance(start, goal), 0, start))

    frames = [new_maze.copy()]
    visited_count = 0
    path_length = 0

    t0 = time.time()
    while heap:
        f, g, (r, c) = heapq.heappop(heap)
        if (r, c) == goal:
            cur = goal
            while cur != start:
                if new_maze[cur] not in [2, 3]:
                    new_maze[cur] = 5  # Path=Blue
                    frames.append(new_maze.copy())
                cur = parent[cur]
                path_length += 1
            t1 = time.time()
            exec_time = t1 - t0
            print(f"Visited nodes: {visited_count}")
            print(f"Path length: {path_length}")
            print(f"Execution time: {exec_time:.5f} seconds")
            return frames, True

        if (r, c) in visited:
            continue
        visited.add((r, c))
        visited_count += 1

        if new_maze[r, c] not in [2, 3]:
            new_maze[r, c] = 4  # Visited=Green
            frames.append(new_maze.copy())

        for nr, nc in get_neighbors(maze, r, c):
            if (nr, nc) not in visited:
                parent[(nr, nc)] = (r, c)
                g_new = g + 1
                f_new = g_new + manhattan_distance((nr, nc), goal)
                heapq.heappush(heap, (f_new, g_new, (nr, nc)))

    t1 = time.time()
    exec_time = t1 - t0
    print("No path found.")
    print(f"Visited nodes: {visited_count}")
    print(f"Execution time: {exec_time:.5f} seconds")
    return frames, False


In [ ]:

maze_with_sg = place_start_goal(maze)
start = (1, 1)
goal = (maze.shape[0]-2, maze.shape[1]-2)

frames, found = a_star_animated_logging(maze_with_sg, start, goal)